In [ ]:
# This notebook is for creating and testing method on curating datasets on stock trading data for offline reinforcement learning with decision transformer model
# This will first create a custom dataset class that can provide sliding window data
# Then it will use those datasets to create gym environments and sample state, action, reward which then store as a replay buffer
# Group these replay buffers and export as a dataset

In [1]:
# import helper function for getting stock data
from getstock import get_stock_data_yf_between_with_indicators

# get stock data with technical indicators
stock_name = 'AAPL'
start_date = '2019-01-01'
end_date = '2021-12-31'
interval = '1d'
indicators = ['volume_obv', 'trend_macd', 'momentum_rsi']

stockdata = get_stock_data_yf_between_with_indicators(stock_name, start_date, end_date, interval, indicators)


[*********************100%***********************]  1 of 1 completed


/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/ta/trend.py:780: RuntimeWarning: invalid value encountered in double_scalars
  dip[idx] = 100 * (self._dip[idx] / value)
/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/ta/trend.py:785: RuntimeWarning: invalid value encountered in double_scalars
  din[idx] = 100 * (self._din[idx] / value)


In [2]:
# create a custom dataset class that can provide sliding window data
import torch
from torch.utils.data import Dataset

class SlidingDataset(Dataset):
    def __init__(self, data, window_size, stride=1, transform=None):
        self.data = data
        self.window_size = window_size
        self.stride = stride
        self.transform = transform

    def __len__(self):
        return (len(self.data) - self.window_size) // self.stride + 1

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        start = idx * self.stride
        end = start + self.window_size
        sample = self.data[start:end]

        if self.transform:
            sample = self.transform(sample)

        return sample

/home/victoru/anaconda3/envs/testpython/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
from datasets import Dataset as huggingfaceDataset

ModuleNotFoundError: No module named 'datasets'

In [3]:
# create a sliding dataset and its dataloader
window_size = 30
stride = 1
dataset = SlidingDataset(stockdata, window_size, stride)



In [4]:
# helper function to add the custom indicators to the environment
def add_signals(env):
    start = env.frame_bound[0] - env.window_size
    end = env.frame_bound[1]
    prices = env.df.loc[:, 'Low'].to_numpy()[start:end]
    signal_features = env.df.loc[:, ['Low','Volume','SMA', 'RSI', 'OBV']].to_numpy()[start:end]
    return prices, signal_features

# create a gym environment from loading sliding dataset and sample state, action, reward
import gym
import gym_anytrading
from gym_anytrading.envs import StocksEnv

# create the custom environment with signal features
class CustomStockEnv(StocksEnv):
    _process_data = add_signals

envwindow_size = 1

# to do - create dataset class to store the state action reward

for i in range(len(dataset)):
    while True:
        env = CustomStockEnv(df=dataset[i], window_size=envwindow_size, frame_bound=(0, len(dataset[i])))
        env.reset()
        state = env._get_obs()
        action = env.action_space.sample()
        nextstate, reward, done, _ = env.step(action)
        # to do - store state, action, reward, nextstate, done in dataset
        print(_)
        if done:
            break



TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.frame.DataFrame'>

In [ ]:
# create a gym environment for each sliding window and sample state, action, reward
import gym
import gym_anytrading

